In [223]:
# https://wikidocs.net/66747

## 02. 양방향 RNN을 이용한 품사 태깅

In [224]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchtext.legacy import data
from torchtext.legacy import datasets
import time
import random

In [225]:
SEED = 1234
random.seed(SEED)
torch.manual_seed(SEED)

In [226]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

### 2. 훈련 데이터에 대한 이해

In [227]:
# 이번에도 토치 텍스트를 이용해보자

# 1. 필드 정의하기
# 이번에 사용할 데이터는 총 3개의 열, 즉 3개의 필드를 갖고 있다. 레이블은 총 2개.
# 이중 1개만 사용할 것인데 원만하게 데이터를 불러오기 위해 우선 3개 모두 필드를 정의해주자

In [228]:
# 3개의 필드 정의
TEXT = data.Field(lower = True) 
UD_TAGS = data.Field(unk_token = None)
PTB_TAGS = data.Field(unk_token = None)

fields = (('text', TEXT), ('udtags', UD_TAGS), ('ptbtags', PTB_TAGS))

In [229]:
# 2. 데이터셋 만들기
# 토치텍스트에서 제공하는 훈련 데이터를 불러오는 동시에 데이터셋을 만들어보자. 훈련/검증/테스트로 나누자
# UDPOS(SequenceTaggingDataset)
train_data, valid_data, test_data = datasets.UDPOS.splits(fields)

In [230]:
# 훈련/검증/테스트의 크기를 확인
print(f'훈련 샘플의 개수: {len(train_data)}')
print(f'검증 샘플의 개수: {len(valid_data)}')
print(f'테스트 샘플의 개수: {len(test_data)}')

훈련 샘플의 개수: 12543
검증 샘플의 개수: 2002
테스트 샘플의 개수: 2077


In [231]:
# 데이터셋을 생성했으니 훈련 데이터의 필드들을 출력해서 확인 해보자
print(train_data.fields)

{'text': <torchtext.legacy.data.field.Field object at 0x0000021E32FA2BE0>, 'udtags': <torchtext.legacy.data.field.Field object at 0x0000021E32FA25B0>, 'ptbtags': <torchtext.legacy.data.field.Field object at 0x0000021E3301CAF0>}


In [232]:
# 훈련 데이터의 첫번째 샘플에서 text와 두 개의 레이블을 모두 출력해보자

# 첫번째 훈련 샘플의 text 필드
print(vars(train_data.examples[0])['text'])
# 이 필드에는 2개의 레이블이 있다. 사용할 것이 udtags 이고 사용하지 않을 것이 ptbtags 이다.
print(vars(train_data.examples[0])['udtags'])
print(vars(train_data.examples[0])['ptbtags'])


['al', '-', 'zaman', ':', 'american', 'forces', 'killed', 'shaikh', 'abdullah', 'al', '-', 'ani', ',', 'the', 'preacher', 'at', 'the', 'mosque', 'in', 'the', 'town', 'of', 'qaim', ',', 'near', 'the', 'syrian', 'border', '.']
['PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'ADJ', 'NOUN', 'VERB', 'PROPN', 'PROPN', 'PROPN', 'PUNCT', 'PROPN', 'PUNCT', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'DET', 'NOUN', 'ADP', 'PROPN', 'PUNCT', 'ADP', 'DET', 'ADJ', 'NOUN', 'PUNCT']
['NNP', 'HYPH', 'NNP', ':', 'JJ', 'NNS', 'VBD', 'NNP', 'NNP', 'NNP', 'HYPH', 'NNP', ',', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'DT', 'NN', 'IN', 'NNP', ',', 'IN', 'DT', 'JJ', 'NN', '.']


In [233]:
# 3. 단어 집합(vocabulary) 만들기
# 이제 단어 집합을 생성해보자
# 그리고 단어 집합을 생성 시에 사전 훈련된 워드 임베딩인 Glove을 사용해보자

# 최소 허용 빈도
MIN_FREQ = 5

# 사전 훈련된 워드 임베딩 Glove 다운로드
TEXT.build_vocab(train_data, min_freq = MIN_FREQ, vectors = "glove.6B.100d")
UD_TAGS.build_vocab(train_data)
PTB_TAGS.build_vocab(train_data)

In [234]:
# 상위 빈도수 20개 단어를 확인해보자
print(TEXT.vocab.freqs.most_common(20))

# 영어는 기본적으로 the의 빈도수가 가장 많다.
# 토치텍스트는 빈도수가 가장 높은 단어부터 작은 숫자를 부여한다.
# 물론 <unk>는 0번, <pad>는 1번으로 자동 부여하고 시작한다.

[('the', 9076), ('.', 8640), (',', 7021), ('to', 5137), ('and', 5002), ('a', 3782), ('of', 3622), ('i', 3379), ('in', 3112), ('is', 2239), ('you', 2156), ('that', 2036), ('it', 1850), ('for', 1842), ('-', 1426), ('have', 1359), ('"', 1296), ('on', 1273), ('was', 1244), ('with', 1216)]


In [235]:
# 상위 정수 인덱스를 가진 10개의 단어를 확인해보자. 다시 말해 0~9번까지의 단어를 확인해보자
print(TEXT.vocab.itos[:10])

['<unk>', '<pad>', 'the', '.', ',', 'to', 'and', 'a', 'of', 'i']


In [236]:
# 단어가 몇번 나왔는지 빈도수까지 확인하자
print(TEXT.vocab.freqs.most_common(10))

[('the', 9076), ('.', 8640), (',', 7021), ('to', 5137), ('and', 5002), ('a', 3782), ('of', 3622), ('i', 3379), ('in', 3112), ('is', 2239)]


In [237]:
# 이번에는 레이블의 단어 집합에 대해 빈도수가 가장 높은 단어들과 그 빈도수를 출력해보자

# 상위 빈도순으로 udtags 출력
print(UD_TAGS.vocab.freqs.most_common())

# 전부 출력한 결과가 아래이다. 결과로 보았을 때 형태소를 구분하려 레이블을 나누었다.

[('NOUN', 34781), ('PUNCT', 23679), ('VERB', 23081), ('PRON', 18577), ('ADP', 17638), ('DET', 16285), ('PROPN', 12946), ('ADJ', 12477), ('AUX', 12343), ('ADV', 10548), ('CCONJ', 6707), ('PART', 5567), ('NUM', 3999), ('SCONJ', 3843), ('X', 847), ('INTJ', 688), ('SYM', 599)]


In [238]:
# UD_TAGS 의 상위 정수 인덱스 10개 단어를 확인해보자
print(UD_TAGS.vocab.itos[:10])

['<pad>', 'NOUN', 'PUNCT', 'VERB', 'PRON', 'ADP', 'DET', 'PROPN', 'ADJ', 'AUX']


In [239]:
# 레이블에 속한 단어들의 분포를 확인하자

# 태그 레이블의 분포를 확인하는 함수. tag_counts에 UD_TAGS.vocab.freqs.most_common() 이 들어갈 것이다.
def tag_percentage(tag_counts):
    total_count = sum([count for tag, count in tag_counts])
    tag_counts_percentages  = [(tag, count, count/total_count) for tag, count in tag_counts]

    return tag_counts_percentages

In [240]:
print('Tag Occurences Percentage\n')

for tag, count, percent in tag_percentage(UD_TAGS.vocab.freqs.most_common()):
    print(f'{tag}\t{count}\t{percent*100:4.1f}%')

Tag Occurences Percentage

NOUN	34781	17.0%
PUNCT	23679	11.6%
VERB	23081	11.3%
PRON	18577	 9.1%
ADP	17638	 8.6%
DET	16285	 8.0%
PROPN	12946	 6.3%
ADJ	12477	 6.1%
AUX	12343	 6.0%
ADV	10548	 5.2%
CCONJ	6707	 3.3%
PART	5567	 2.7%
NUM	3999	 2.0%
SCONJ	3843	 1.9%
X	847	 0.4%
INTJ	688	 0.3%
SYM	599	 0.3%


In [241]:
# 4. 데이터로더 만들기

# 이제 데이터로더를 만들자. 배치 크기는 64로 한다

BATCH_SIZE = 64

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE,
    device = device)

In [242]:
# 첫번째 미니 배치만 꺼내서 미니 배치의 구성, 크기, text를 출력해보자
batch = next(iter(train_iterator))
batch


[torchtext.legacy.data.batch.Batch of size 64 from UDPOS]
	[.text]:[torch.cuda.LongTensor of size 46x64 (GPU 0)]
	[.udtags]:[torch.cuda.LongTensor of size 46x64 (GPU 0)]
	[.ptbtags]:[torch.cuda.LongTensor of size 46x64 (GPU 0)]

In [243]:
# 첫 번째 미니 배치의 text의 크기를 출력해보자
batch.text.shape

# 이 크기는 [시퀀스 길이, 배치 크기] 이다.
# batch_first = True 하지 않아서 배치 크기가 두번째 차원으로 들어간다

torch.Size([46, 64])

In [244]:
# 첫번째 미니 배치의 text를 확인해보자
batch.text

tensor([[ 732,  167,    2,  ...,    2,   59,  668],
        [  16,  196,  133,  ..., 2991,   46,    1],
        [   1,   29,   48,  ..., 1582,   12,    1],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]], device='cuda:0')

In [245]:
# 3. 모델 구현하기

# 이제 모델을 구현하자. 기본적으로 다대다 RNN을 사용하자
# 일단 양방향 여부와 층의 개수는 변수로 둔다
# 이번 모델에서는 batch_first=True를 사용하지 않았으므로 배치 차원이 맨 앞이 아니다

class RNNPOSTagger(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, bidirectional, dropout): 
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, num_layers = n_layers, bidirectional = bidirectional)
        self.fc = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)        
        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        # text = [sent len, batch size]
        embedded = self.dropout(self.embedding(text))

        # embedded = [sent len, batch size, emb dim]
        outputs, (hidden, cell) = self.rnn(embedded)

        # output = [sent len, batch size, hid dim * n directions]
        # hidden/cell = [n layers * n directions, batch size, hid dim]
        predictions = self.fc(self.dropout(outputs))

        # predictions = [sent len, batch size, output dim]
        return predictions

In [246]:
# 실제 클래스로부터 모델 객체로 생성 시에 양방향 여부 True, 층 개수 2 로 지정하자

INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 128
OUTPUT_DIM = len(UD_TAGS.vocab)
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.25

model = RNNPOSTagger(INPUT_DIM, 
                     EMBEDDING_DIM, 
                     HIDDEN_DIM, 
                     OUTPUT_DIM, 
                     N_LAYERS, 
                     BIDIRECTIONAL, 
                     DROPOUT)

In [247]:
# 파라미터 개수를 확인해보자

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 1,027,510 trainable parameters


In [248]:
# 4. 사전 훈현된 워드 임베딩 사용하기

# 사전 훈련된 워드 임베딩인 Glove를 사용하자
# 이를 위해 토치텍스트의 단어 집합 생성 시에 저장해두었던 Glove 임베딩을 nn.Embedding()에 연결해야 한다
# 우선 단어 집합의 단어들에 맵핑된 사전 훈련된 워드 임베딩을 확인해보자

pretrained_embeddings = TEXT.vocab.vectors
print(pretrained_embeddings.shape)
# 단어 집합에 존재하는 총 3,921개의 단어에 대해 100 차원의 벡터가 맵핑되어져 있다.
# 이제 이를 nn.Embedding()에 연결해주자

torch.Size([3921, 100])


In [249]:
model.embedding.weight.data.copy_(pretrained_embeddings)
# 임데팅 벡터값을 복사

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1020,  0.7700,  0.1169,  ..., -0.1416, -0.1932, -0.4225],
        [-0.0263,  0.0179, -0.5016,  ..., -0.8688,  0.9409, -0.2882],
        [ 0.1519,  0.4712,  0.0895,  ..., -0.4702, -0.3127,  0.1078]])

In [250]:
# 우선 <unk> 와 <pad> 토큰의 인덱스를 저장해두자
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

print(UNK_IDX)
print(PAD_IDX)

0
1


In [251]:
# 임의로 0번, 1번 단어에 0벡터를 만든다

# 0번 임베딩 벡터에 0값 채우기
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
# 1번 임베딩 벡터에 0값 채우기
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

print(model.embedding.weight.data)
# 위 두 줄은 0값이다.

tensor([[ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.1020,  0.7700,  0.1169,  ..., -0.1416, -0.1932, -0.4225],
        [-0.0263,  0.0179, -0.5016,  ..., -0.8688,  0.9409, -0.2882],
        [ 0.1519,  0.4712,  0.0895,  ..., -0.4702, -0.3127,  0.1078]])


In [252]:
# 5.옵티마이저와 비용 함수 구현

# 옵티마이저 설계 전에 레이블 데이터의 패딩 토큰의 인덱스도 확인하자
TAG_PAD_IDX = UD_TAGS.vocab.stoi[UD_TAGS.pad_token]
print(TAG_PAD_IDX)

# 0으로 나오는데 이유는 아래 비용 함수를 선택할 때 인자로 주기 위함이다.

0


In [253]:
# Adam으로 옵티마이저를 설정
optimizer = optim.Adam(model.parameters())

In [254]:
# 비용 함수는 crossentropy 함수
# 이때 레이블 데이터의 패딩 코튼은 비용 함수의 연산에 포함되지 않도록 ignore_index로 설정한다

criterion = nn.CrossEntropyLoss(ignore_index=TAG_PAD_IDX)

In [255]:
# 현재 GPU를 사용 중일때 GPU 연산을 할 수 있도록 지정
model = model.to(device)
criterion = criterion.to(device)

In [256]:
# 아직 훈련되지 않은 모델이지만 모델에 입력값을 넣어 출력(예측값)의 크기를 확인하자
prediction = model(batch.text)
prediction.shape

# 46 = 첫번째 배치의 시퀀스 길이, 첫번째 배치의 길이일 뿐 다른 배치들은 다를 수 있다
# 64 = 배치 크기
# 18 = 레이블 단어장의 크기

torch.Size([46, 64, 18])

In [257]:
# 예측값에 대해서 시퀀스 길이와 배치 길이를 모두 펼쳐주자
prediction = prediction.view(-1, prediction.shape[-1])
prediction.shape

torch.Size([2944, 18])

In [258]:
# 첫번째 배치의 레이블 데이터의 크기를 보자
batch.udtags.shape

torch.Size([46, 64])

In [259]:
# 이를 펼쳐보자
batch.udtags.view(-1).shape

# 2944로 정확한 사이즈를 지정해주었다.

torch.Size([2944])

In [260]:
# 6. 훈련과 평가하기

def categorical_accuracy(preds, y, tag_pad_idx):
    """
    미니 배치에 대한 정확도 출력
    """
    max_preds = preds.argmax(dim = 1, keepdim = True) # get the index of the max probability
    non_pad_elements = (y != tag_pad_idx).nonzero()
    correct = max_preds[non_pad_elements].squeeze(1).eq(y[non_pad_elements])
    return correct.sum() / torch.FloatTensor([y[non_pad_elements].shape[0]])

In [261]:
def train(model, iterator, optimizer, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:

        text = batch.text
        tags = batch.udtags

        optimizer.zero_grad()

        #text = [sent len, batch size]     
        predictions = model(text)

        #predictions = [sent len, batch size, output dim]
        #tags = [sent len, batch size]
        predictions = predictions.view(-1, predictions.shape[-1]) # #predictions = [sent len * batch size, output dim]
        tags = tags.view(-1) # tags = [sent len * batch_size]

        #predictions = [sent len * batch size, output dim]
        #tags = [sent len * batch size]
        loss = criterion(predictions, tags)

        acc = categorical_accuracy(predictions, tags, tag_pad_idx)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [262]:
def evaluate(model, iterator, criterion, tag_pad_idx):
    
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():

        for batch in iterator:

            text = batch.text
            tags = batch.udtags

            predictions = model(text)

            predictions = predictions.view(-1, predictions.shape[-1])
            tags = tags.view(-1)

            loss = criterion(predictions, tags)

            acc = categorical_accuracy(predictions, tags, tag_pad_idx)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [263]:
N_EPOCHS = 10

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion, TAG_PAD_IDX)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion, TAG_PAD_IDX)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')

    print(f'Epoch: {epoch+1:02}')
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
test_loss, test_acc = evaluate(model, test_iterator, criterion, TAG_PAD_IDX)

print(f'Test Loss: {test_loss:.3f} |  Test Acc: {test_acc*100:.2f}%')

RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!